In [1]:
# Model Training
from datasets import load_from_disk

datasets = load_from_disk("./encoded_data2_better")
datasets


c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Threats and Security on the Internet', '__index_level_0__'],
        num_rows: 350
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Threats and Security on the Inte

In [2]:
model_labels = [label for label in datasets['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [11]:
datasets["train"].column_names

['Unnamed: 0',
 'String',
 'Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Threats and Security on the Internet',
 '__index_level_0__']

In [12]:
datasets["train"].features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'String': Value(dtype='string', id=None),
 'Algorithm Design': Value(dtype='int64', id=None),
 'Basic Machine Organisation': Value(dtype='int64', id=None),
 'Computer System': Value(dtype='int64', id=None),
 'Data Manipulation and Analysis': Value(dtype='int64', id=None),
 'Data Organisation and Data Control': Value(dtype='int64', id=None),
 'Elementary Web Authoring': Value(dtype='int64', id=None),
 'Health and Ethical Issues': Value(dtype='int64', id=None),
 'Information Processing': Value(dtype='int64', id=None),
 'Intellectual Property': Value(dtype='int64', id=None),
 'Internet Services and Applications': Value(dtype='int64', id=None),
 'Multimedia Elements': Value(dtype='int64', id=None),
 'Networking and Internet Basics': Value(dtype='int64', id=None),
 'Program Development': Value(dtype='int64', id=None),
 'Threats and Security on the Internet': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [10]:
print(f"{datasets.train.column_names}\n{datasets.train.features}")

AttributeError: 'DatasetDict' object has no attribute 'train'

In [3]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Threats and Security on the Internet']

In [4]:
id2label = {idx:label for idx, label in enumerate(model_labels)}
label2id = {label:idx for idx, label in enumerate(model_labels)}

In [15]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import numpy as np

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [17]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in model_labels}
    labels_matrix = np.zeros((len(text), len(model_labels)))
    for idx, label in enumerate(model_labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [18]:
# tokenize the dataset
encoded_ds = datasets.map(encode_data, batched=True, remove_columns=datasets['train'].column_names)

Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data2_better\train\cache-6fd00cfa85b31aec.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data2_better\valid\cache-9f8ae3e78e953f1d.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data2_better\test\cache-28791c87e7a459c1.arrow


In [20]:
encoded_ds['train']['input'][0]

[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [9]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Basic Machine Organisation']

In [10]:
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [11]:
len(model_labels)

14

In [12]:
encoded_ds["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 350
})

In [14]:
# len(train_dataloader)

In [15]:

# define model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=len(model_labels), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [16]:
# from torch.optim import AdamW

# optimizer = AdamW(model.parameters(), lr=2e-5)

In [17]:
# from transformers import get_scheduler

# num_epochs = 70
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
# )

In [18]:
# train model
batch_size = 8
metric_name = "f1"

In [19]:
import evaluate
metric  = evaluate.load("accuracy")

In [20]:
import time

train_name_dir = time.strftime(f"%Y%m%d_%H%M%S$-train-model-{model.name_or_path}")
train_name_dir

'20230203_083620$-train-model-distilbert-base-uncased'

In [21]:
from transformers import TrainingArguments, Trainer


args = TrainingArguments(
    output_dir="./distBert-v4/"+ train_name_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 70,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='./distBert-v4/logging/'+ train_name_dir
)

In [22]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction


def multi_label_metrics(predictions, labels, threshold=0.5):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # logits, labels = predictions
    # prediction = np.argmax(logits, axis=-1)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

def compute_metrics_new(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [24]:
# model.to(device)

In [25]:
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7023, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0815,  0.1147, -0.0566,  0.0512,  0.0432, -0.0413, -0.0578,  0.0303,
          0.0477, -0.0607,  0.0151, -0.0045,  0.0462,  0.0851]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [27]:
trainer.train()

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 350
  Num Epochs = 70
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3080
  Number of trainable parameters = 66964238
  1%|▏         | 42/3080 [00:03<02:24, 20.98it/s] ***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
                                                 
  1%|▏         | 44/3080 [00:03<02:24, 20.98it/s]Saving model checkpoint to ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-44
Configuration saved in ./distBert-v4/20230203_

{'eval_loss': 0.3635549247264862, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.0754, 'eval_samples_per_second': 583.483, 'eval_steps_per_second': 79.566, 'epoch': 1.0}


Model weights saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-44\pytorch_model.bin
tokenizer config file saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-44\tokenizer_config.json
Special tokens file saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-44\special_tokens_map.json
  3%|▎         | 87/3080 [00:06<02:22, 20.94it/s]***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
                                                 
  3%|▎         | 88/3080 [00:06<02:22, 20.94it/s]Saving model checkpoint to ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-88
Configuration saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-88\config.json


{'eval_loss': 0.2967499792575836, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.0714, 'eval_samples_per_second': 616.577, 'eval_steps_per_second': 84.079, 'epoch': 2.0}


Model weights saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-88\pytorch_model.bin
tokenizer config file saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-88\tokenizer_config.json
Special tokens file saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-88\special_tokens_map.json
  4%|▍         | 132/3080 [00:09<02:19, 21.08it/s]***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
                                                  
  4%|▍         | 132/3080 [00:09<02:19, 21.08it/s]Saving model checkpoint to ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-132
Configuration saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-132\config.json


...

 99%|█████████▊| 3036/3080 [13:06<00:02, 19.83it/s]Model weights saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-3036\pytorch_model.bin
tokenizer config file saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-3036\tokenizer_config.json
Special tokens file saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-3036\special_tokens_map.json
100%|██████████| 3080/3080 [13:20<00:00, 19.04it/s]***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
                                                   
100%|██████████| 3080/3080 [13:20<00:00, 19.04it/s]Saving model checkpoint to ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-3080
Configuration saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-3080\config.json


{'eval_loss': 0.16809970140457153, 'eval_f1': 0.7252747252747251, 'eval_roc_auc': 0.811988543371522, 'eval_accuracy': 0.5681818181818182, 'eval_runtime': 0.0812, 'eval_samples_per_second': 541.728, 'eval_steps_per_second': 73.872, 'epoch': 70.0}


Model weights saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-3080\pytorch_model.bin
tokenizer config file saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-3080\tokenizer_config.json
Special tokens file saved in ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-3080\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./distBert-v4/20230203_083620$-train-model-distilbert-base-uncased\checkpoint-1892 (score: 0.7472527472527474).
100%|██████████| 3080/3080 [13:27<00:00,  3.81it/s]

{'train_runtime': 807.9284, 'train_samples_per_second': 30.324, 'train_steps_per_second': 3.812, 'train_loss': 0.04587776635761385, 'epoch': 70.0}


TrainOutput(global_step=3080, training_loss=0.04587776635761385, metrics={'train_runtime': 807.9284, 'train_samples_per_second': 30.324, 'train_steps_per_second': 3.812, 'train_loss': 0.04587776635761385, 'epoch': 70.0})

In [28]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 44
  Batch size = 8
100%|██████████| 6/6 [00:00<00:00, 89.23it/s]


{'eval_loss': 0.16147714853286743,
 'eval_f1': 0.7472527472527474,
 'eval_roc_auc': 0.8224904528096018,
 'eval_accuracy': 0.5909090909090909,
 'eval_runtime': 0.0792,
 'eval_samples_per_second': 555.433,
 'eval_steps_per_second': 75.741,
 'epoch': 70.0}

In [30]:
prediction_result = trainer.predict(encoded_ds['test'])

***** Running Prediction *****
  Num examples = 44
  Batch size = 8
100%|██████████| 6/6 [00:00<00:00, 109.13it/s]


In [31]:
print(prediction_result.metrics)

{'test_loss': 0.13981007039546967, 'test_f1': 0.6976744186046512, 'test_roc_auc': 0.799949609473419, 'test_accuracy': 0.5909090909090909, 'test_runtime': 0.1865, 'test_samples_per_second': 235.964, 'test_steps_per_second': 32.177}


In [32]:
trainer.save_model("./distlBert-model-v4")

Saving model checkpoint to ./distlBert-model-v4
Configuration saved in ./distlBert-model-v4\config.json
Model weights saved in ./distlBert-model-v4\pytorch_model.bin
tokenizer config file saved in ./distlBert-model-v4\tokenizer_config.json
Special tokens file saved in ./distlBert-model-v4\special_tokens_map.json
